<a href="https://colab.research.google.com/github/mzohaibnasir/finetuningLLM/blob/main/02_QLoRA%26LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QLoRA and LoRA Quantization